# Users - Post-processing Data Transformations and Exploratory Data Analysis

The present notebook is part of a series of notebooks related to the MSc. thesis: **Sentiment analysis on generative language models based on Social Media commentary of industry participants**

The MSc. thesis research was conducted based on tweets about ChatGPT. These were collected, processed and analyzed with the scope of answering the following research question:

**How are generative language models perceived by participants of different industries based on social media commentary?**

For answering the research question, a focal point consists in analyzing the twitter data already processed by the means of LDA for topic modeling and VADER for sentiment analysis. 

To explore the users from a longitudinal perspective, throughout this notebook, the data will be transformed so that it will be most fit for the analysis conducted in PowerBI. Additionally, certain complementary visualizations and calculations will also be conducted in this notebook.

In [1]:
#Import packages cell
import pandas as pd
from pandas import option_context
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import math
from IPython.display import clear_output


import plotly.express as px

#Set pandas options for ease of cleaning
pd.set_option('display.max_rows', 500)
pd.options.mode.chained_assignment = None

# Condesned data

This section serves to loading the data for further processing.

In [14]:
#Load Tweets Information
tweets_information = pd.read_csv("data_files/13_tweets_data_condensed_FINAL.csv", parse_dates=['tweet_date'])

tweets_information['tweet_id'] = tweets_information['tweet_id'].astype(str)
tweets_information['usr_userid'] = tweets_information['usr_userid'].astype(str)
tweets_information['tweet_inreplytotweetid'] = tweets_information['tweet_inreplytotweetid'].apply(lambda x: str('%.0f' % x))
tweets_information['usr_inreply_id'] = tweets_information['usr_inreply_id'].apply(lambda x: str('%.0f' % x))
tweets_information['date'] = tweets_information['tweet_date'].dt.date

for column in ['tweet_hashtags','tweet_content_tokens','vader_polarity']:
    tweets_information[column] = tweets_information[column].apply(lambda x: ast.literal_eval(x) if str(x)[0]=="[" or str(x)[0]=="{" else [])

tweets_information.head(3)

,tweet_id,tweet_content_sentiment_prepro,vader_polarity,sentiment,compound,tweet_content_clean,tweet_content_tokens,tweet_topic,tweet_date,tweet_content,tweet_replycount,tweet_retweetcount,tweet_likecount,tweet_inreplytotweetid,tweet_hashtags,usr_userid,usr_inreply_id,tweet_relevant,new_user,date
0,1598014056790622225,ChatGPT: Optimizing Language Models for Dialogue,"{'neg': 0.0, 'neu': 0.625, 'pos': 0.375, 'comp...",Positive,0.4588,chatgpt optimizing language models for dialogue,"[optimize, dialogue, language, model]",UnclearTopic,2022-11-30 18:00:15+00:00,ChatGPT: Optimizing Language Models for Dialog...,0,0,2,nan,[],354863991,nan,True,True,2022-11-30
1,1598014522098208769,"Try talking with ChatGPT, our new AI system wh...","{'neg': 0.0, 'neu': 0.664, 'pos': 0.336, 'comp...",Positive,0.8225,try talking with chatgpt our new ai system whi...,"[feedback, talk, improve, optimize, system, di...",UnclearTopic,2022-11-30 18:02:06+00:00,"Try talking with ChatGPT, our new AI system wh...",1320,3518,13703,nan,[],4398626122,nan,True,True,2022-11-30
2,1598015627540635648,"Just launched ChatGPT, our new AI system which...","{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",Positive,0.5423,just launched chatgpt our new ai system which ...,"[optimize, system, dialogue, new, launch]",UnclearTopic,2022-11-30 18:06:29+00:00,"Just launched ChatGPT, our new AI system which...",84,369,2370,nan,[],162124540,nan,True,True,2022-11-30


In [12]:
#tweets_information.to_csv("tweets_data_condensed_FINAL.csv",index=False)

In [19]:
# Load user information
user_information= pd.read_csv("data_files/14_users_data_condensed_FINAL.csv")
user_information['usr_userid'] = user_information['usr_userid'].astype(str)
user_information.rename(columns={'date':'join_date'},inplace=True)

for column in ['usr_description_hashtags','usr_description_mentions','usr_description_URLs','usr_description_tokens']:
    user_information[column] = user_information[column].apply(lambda x: ast.literal_eval(x) if str(x)[0]=="[" or str(x)[0]=="{" else [])

user_information.head(3)

,usr_userid,usr_description_origin,usr_description_hashtags,usr_description_mentions,usr_description_URLs,usr_description_emoji,usr_description_clean,usr_description_tokens,urs_industry_categ,usr_location,first_tweet_date,average_compound,sentiment
0,354863991,using A.I. to propel the real estate industry ...,"[PropTech, AI]",[],[],NaN,using ai to propel the real estate industry fo...,"[cle, sport, enjoy, forward, industry, read, p...",Arts & Entertainment,"Cleveland, OH",2022-11-30,0.253567,Positive
1,4398626122,OpenAI’s mission is to ensure that artificial ...,[],[],[openai.com/jobs],NaN,openais mission is to ensure that artificial g...,"[mission, hire, intelligence, artificial, huma...",NotIndustry(1),NaN,2022-11-30,0.387829,Positive
2,162124540,President & Co-Founder @OpenAI,[],[OpenAI],[],NaN,president co-founder,"[co, president, founder]",Entrepreneurship & Business Leadership,NaN,2022-11-30,0.470832,Positive


In [5]:
user_information.urs_industry_categ.unique()

array(['Arts & Entertainment', 'NotIndustry(1)',
       'Entrepreneurship & Business Leadership', 'Software Engineering',
       'NotIndustry(2)', 'Marketing & Business Development',
       'Education & Academia', 'Finance & Crypto', 'NotSpecified'],
      dtype=object)

In [63]:
#user_information.replace({'urs_industry_categ' : { 'Entreprenourship & Business Commentary' : 'Entrepreneurship & Business Leadership', 'Marketing and Business Development' : 'Marketing & Business Development', 'Software Engingeering' : 'Software Engineering' }},inplace=True)
#user_information.to_csv("users_data_condensed_FINAL.csv",index=False)

# Transform data to daily by user

In this section, a dataframe holding user specific information on a daily basis, will be created

In [15]:
#Filter for relevant tweets
tweets_info = tweets_information[tweets_information['tweet_relevant']==True]

In [29]:
#Create function to get all necessary data for each user by date
def user_daily_data(tweets_information,user_information):
    i=0
    for usr_userid in user_information.usr_userid.unique():

        #if usr_userid not in test.usr_user_id.unique()[:-1]:

        clear_output(wait=True)        
        print(f"{datetime.now()} USERID: {usr_userid} \nremianing rows:{user_information.shape[0]-i}")   

        filtered_by_user_df =  tweets_information[tweets_information['usr_userid']== usr_userid]

        for date in filtered_by_user_df.date.unique():

            dict_user = {'date':[],
                        'usr_userid':[],
                        
                        'total_tweets':[],
                        'total_positive':[],
                        'total_negative':[],
                        'total_neutral':[],
                        
                        'total_inreply':[],
                        'total_positive_inreply':[],
                        'total_negative_inreply':[],
                        'total_neutral_inreply':[],

                        'avg_compund':[],
                        'avg_positive':[],
                        'avg_negative':[],
                        }

            filtered_by_date_df = filtered_by_user_df[filtered_by_user_df['date']== date]

            dict_user['date'].append(date)
            dict_user['usr_userid'].append(usr_userid)

            total_tweets = filtered_by_date_df.shape[0]
            dict_user['total_tweets'].append(total_tweets)

            positive_df = filtered_by_date_df[filtered_by_date_df['sentiment']=='Positive']
            negative_df = filtered_by_date_df[filtered_by_date_df['sentiment']=='Negative']
            neutral_df = filtered_by_date_df[filtered_by_date_df['sentiment']=='Neutral']

            total_positive = positive_df.shape[0]
            total_negative = negative_df.shape[0]
            total_neutral = neutral_df.shape[0]

            dict_user['total_positive'].append(total_positive)
            dict_user['total_negative'].append(total_negative)
            dict_user['total_neutral'].append(total_neutral)

            total_inreply = filtered_by_date_df[filtered_by_date_df['usr_inreply_id']!='nan'].shape[0]
            dict_user['total_inreply'] = total_inreply

            total_positive_inreply = positive_df[positive_df['usr_inreply_id']!='nan'].shape[0]
            total_negative_inreply = negative_df[negative_df['usr_inreply_id']!='nan'].shape[0]
            total_neutral_inreply = neutral_df[neutral_df['usr_inreply_id']!='nan'].shape[0]

            dict_user['total_positive_inreply'].append(total_positive_inreply)
            dict_user['total_negative_inreply'].append(total_negative_inreply)
            dict_user['total_neutral_inreply'].append(total_neutral_inreply)

            dict_user['avg_compund'].append(filtered_by_date_df['compound'].mean())
            dict_user['avg_positive'].append(positive_df['compound'].mean())
            dict_user['avg_negative'].append(negative_df['compound'].mean())


            pd.DataFrame.from_dict(dict_user).to_csv("data_files/15_daily_user_data.csv", mode='a', index=False, header=False)
            i=i+1



In [ ]:
#Run function
user_daily_data(tweets_info, user_information)

In [4]:
#Read resulting df
cols = ['date','usr_userid','total_tweets','total_positive',
                                                'total_negative','total_neutral','total_inreply','total_positive_inreply',
                                                'total_negative_inreply','total_neutral_inreply','avg_compund',
                                                'avg_positive', 'avg_negative' ]

test =pd.read_csv("data_files/daily_user_data.csv", names=cols, header=None)
test

,date,usr_user_id,total_tweets,total_positive,total_negative,total_neutral,total_inreply,total_positive_inreply,total_negative_inreply,total_neutral_inreply,avg_compund,avg_positive,avg_negative
0,2022-11-30,354863991,1,1,0,0,0,0,0,0,0.4588,0.4588,NaN
1,2023-01-16,354863991,1,1,0,0,0,0,0,0,0.8126,0.8126,NaN
2,2023-02-01,354863991,2,1,0,1,0,0,0,0,0.1250,0.2500,NaN
3,2023-03-23,354863991,1,0,0,1,0,0,0,0,0.0000,NaN,NaN
4,2023-03-24,354863991,1,0,0,1,0,0,0,0,0.0000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609279,2023-04-07,2469116083,1,1,0,0,1,1,0,0,0.2500,0.2500,NaN
1609280,2023-04-07,1570786718017617921,1,0,1,0,0,0,0,0,-0.2960,NaN,-0.2960
1609281,2023-04-07,1644445633128833025,3,0,0,3,0,0,0,0,0.0000,NaN,NaN
1609282,2023-04-07,1625106180090757120,1,0,1,0,1,0,1,0,-0.3818,NaN,-0.3818


In [13]:
#Check if all users were processed
test.usr_user_id.nunique()

698701

In [22]:
#Assign polarity
for i in range(test.shape[0]):
    compound = test.at[i,'avg_compund']
    
    if compound > 0.05:
        test.at[i,'sentiment'] = "Positive"
    elif compound < -0.05:
        test.at[i,'sentiment'] = "Negative"
    else:
        test.at[i,'sentiment'] = "Neutral"

In [16]:
#test.drop_duplicates().to_csv("daily_user_data.csv",index=False)

# Reading daily user data

In [31]:
#Read data
daily_user_data =pd.read_csv("data_files/15_daily_user_data.csv")
daily_user_data['usr_userid']=daily_user_data['usr_userid'].astype(str)

daily_user_data=daily_user_data.drop(columns="index")

daily_user_data.head(3)

,date,usr_userid,total_tweets,total_positive,total_negative,total_neutral,total_inreply,total_positive_inreply,total_negative_inreply,total_neutral_inreply,avg_compund,avg_positive,avg_negative,sentiment
0,2022-11-30,354863991,1,1,0,0,0,0,0,0,0.4588,0.4588,NaN,Positive
1,2023-01-16,354863991,1,1,0,0,0,0,0,0,0.8126,0.8126,NaN,Positive
2,2023-02-01,354863991,2,1,0,1,0,0,0,0,0.1250,0.2500,NaN,Positive


In [21]:
daily_user_data.usr_userid.nunique()

698701

In [22]:
daily_user_data.drop_duplicates().shape

(1315250, 15)

In [23]:
#Add industry data
daily_user_data=daily_user_data.merge(user_information[['usr_userid','urs_industry_categ']],on='usr_userid',how='left')

In [25]:
#Show how many times on average users change their sentiment
status_change_rate = daily_user_data.groupby(['urs_industry_categ', 'usr_userid'])['sentiment'].count().reset_index().groupby('urs_industry_categ')['sentiment'].mean()
status_change_rate

urs_industry_categ
Arts & Entertainment                      1.612487
Education & Academia                      1.887105
Entrepreneurship & Business Leadership    1.933406
Finance & Crypto                          1.824360
Marketing & Business Development          2.689784
NotIndustry(1)                            1.924863
NotIndustry(2)                            1.632563
NotSpecified                              1.434921
Software Engineering                      2.016107
Name: sentiment, dtype: float64

In [32]:
daily_user_data.head(3)

,date,usr_userid,total_tweets,total_positive,total_negative,total_neutral,total_inreply,total_positive_inreply,total_negative_inreply,total_neutral_inreply,avg_compund,avg_positive,avg_negative,sentiment
0,2022-11-30,354863991,1,1,0,0,0,0,0,0,0.4588,0.4588,NaN,Positive
1,2023-01-16,354863991,1,1,0,0,0,0,0,0,0.8126,0.8126,NaN,Positive
2,2023-02-01,354863991,2,1,0,1,0,0,0,0,0.1250,0.2500,NaN,Positive


In [37]:
users_overall_compound = daily_user_data.groupby('usr_userid')['avg_compund'].mean().reset_index()

In [38]:
users_overall_compound

,usr_userid,avg_compund
0,1000000995814879232,0.457650
1,1000002795213385734,0.000000
2,1000003296562737153,-0.904200
3,1000004500525060097,0.161267
4,1000006344089497600,0.000000
...,...,...
698696,999984092908261376,0.765900
698697,999986210885001217,0.335250
698698,99999577,0.000000
698699,999997132349329408,0.000000


In [40]:
user_information= user_information.drop(columns="average_compound").merge(users_overall_compound,on="usr_userid",how='left')

In [41]:
user_information.head()

,usr_userid,usr_description_origin,usr_description_hashtags,usr_description_mentions,usr_description_URLs,usr_description_emoji,usr_description_clean,usr_description_tokens,urs_industry_categ,usr_location,first_tweet_date,sentiment,avg_compund
0,354863991,using A.I. to propel the real estate industry ...,"[PropTech, AI]",[],[],NaN,using ai to propel the real estate industry fo...,"[cle, sport, enjoy, forward, industry, read, p...",Arts & Entertainment,"Cleveland, OH",2022-11-30,Positive,0.279280
1,4398626122,OpenAI’s mission is to ensure that artificial ...,[],[],[openai.com/jobs],NaN,openais mission is to ensure that artificial g...,"[mission, hire, intelligence, artificial, huma...",NotIndustry(1),NaN,2022-11-30,Positive,0.387829
2,162124540,President & Co-Founder @OpenAI,[],[OpenAI],[],NaN,president co-founder,"[co, president, founder]",Entrepreneurship & Business Leadership,NaN,2022-11-30,Positive,0.438995
3,1573710710852489216,The latest developments in the world of artifi...,[],[],[],NaN,the latest developments in the world of artifi...,"[development, world, artificial, intelligence,...",Software Engineering,Weekly newsletter →,2022-11-30,Positive,0.164097
4,4617024083,I'm a bot. I post articles from the Hacker New...,[],[c17r_],[],NaN,im a bot i post articles from the hacker news ...,"[bounce, ill, post, front, dayby, hacker, bot,...",NotIndustry(2),"NoVA, USA",2022-11-30,Neutral,-0.021681


In [43]:
#Assign polarity
for i in range(user_information.shape[0]):
    compound = user_information.at[i,'avg_compund']
    
    if compound > 0.05:
        user_information.at[i,'sentiment'] = "Positive"
    elif compound < -0.05:
        user_information.at[i,'sentiment'] = "Negative"
    else:
        user_information.at[i,'sentiment'] = "Neutral"

In [44]:
user_information.shape

(698701, 13)

In [45]:
user_information.to_csv("data_files/14_users_data_condensed_FINAL.csv",index=False)

In [46]:
user_information

,usr_userid,usr_description_origin,usr_description_hashtags,usr_description_mentions,usr_description_URLs,usr_description_emoji,usr_description_clean,usr_description_tokens,urs_industry_categ,usr_location,first_tweet_date,sentiment,avg_compund
0,354863991,using A.I. to propel the real estate industry ...,"[PropTech, AI]",[],[],NaN,using ai to propel the real estate industry fo...,"[cle, sport, enjoy, forward, industry, read, p...",Arts & Entertainment,"Cleveland, OH",2022-11-30,Positive,0.279280
1,4398626122,OpenAI’s mission is to ensure that artificial ...,[],[],[openai.com/jobs],NaN,openais mission is to ensure that artificial g...,"[mission, hire, intelligence, artificial, huma...",NotIndustry(1),NaN,2022-11-30,Positive,0.387829
2,162124540,President & Co-Founder @OpenAI,[],[OpenAI],[],NaN,president co-founder,"[co, president, founder]",Entrepreneurship & Business Leadership,NaN,2022-11-30,Positive,0.438995
3,1573710710852489216,The latest developments in the world of artifi...,[],[],[],NaN,the latest developments in the world of artifi...,"[development, world, artificial, intelligence,...",Software Engineering,Weekly newsletter →,2022-11-30,Positive,0.164097
4,4617024083,I'm a bot. I post articles from the Hacker New...,[],[c17r_],[],NaN,im a bot i post articles from the hacker news ...,"[bounce, ill, post, front, dayby, hacker, bot,...",NotIndustry(2),"NoVA, USA",2022-11-30,Neutral,-0.021681
...,...,...,...,...,...,...,...,...,...,...,...,...,...
698696,2469116083,NaN,[],[],[],NaN,NaN,[],NotSpecified,NaN,2023-04-07,Positive,0.250000
698697,1570786718017617921,NaN,[],[],[],NaN,NaN,[],NotSpecified,NaN,2023-04-07,Negative,-0.296000
698698,1644445633128833025,NaN,[],[],[],NaN,NaN,[],NotSpecified,NaN,2023-04-07,Neutral,0.000000
698699,1625106180090757120,NaN,[],[],[],NaN,NaN,[],NotSpecified,NaN,2023-04-07,Negative,-0.381800
